In [101]:
import dgl
import pyspark
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import scipy.sparse as sp
import torch
import pickle
import json
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [102]:
spark = SparkSession.builder.master("local[20]").appName("graphlab_amazon_fraud.com").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 15:36:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 15:36:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/19 15:36:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/19 15:36:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
review_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Category/reviews_Musical_Instruments.json")
meta_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Category/meta_Musical_Instruments.json")

22/10/18 20:26:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
review_df.count()

In [ ]:
review_df.select(F.count_distinct("reviewerID"))

In [ ]:
equals_zero = lambda e: e == 0
rev_df = review_df.withColumn("drop", F.exists(F.col("helpful"), equals_zero))
rev_df = rev_df.filter(F.col("drop") == False).drop(F.col("drop"))
rev_df = rev_df.select("asin", "overall", "reviewText", "reviewerID", "reviewerName", "summary", rev_df.helpful[0], rev_df.helpful[1], "unixReviewTime", "reviewTime")
rev_df

In [ ]:
# rev_df = rev_df.groupBy("reviewerID").agg(F.sum("helpful[0]").alias("helpful_0"), \
#                                 F.sum("helpful[1]").alias("helpful_1"), \
#                                 F.collect_list("asin").alias("asin"), \
#                                 F.collect_list("overall").alias("overall"), \
#                                 F.collect_list("reviewText").alias("reviewText"), \
#                                 F.collect_list("summary").alias("summary"), \
#                                 F.collect_list("reviewerName").alias("reviewerName"), \
#                                 F.collect_list("unixReviewTime").alias("unixReviewTime"), \
#                                 F.collect_list("reviewTime").alias("reviewTime")
#                                 )

In [ ]:
rev_df = rev_df.withColumn("label", (F.col("helpful[0]") / F.col("helpful[1]"))) #.drop("helpful[0]").drop("helpful[1]")
rev_df = rev_df.filter((F.col("label") > 0.8) | (F.col("label") < 0.2))
rev_df = rev_df.withColumn("label", F.udf(lambda a: 1 if a > 0.8 else 0, T.IntegerType())(F.col("label")))

In [ ]:
rev_df = rev_df.groupBy("reviewerID").agg(F.sum("helpful[0]").alias("helpful_0"), \
                                F.sum("helpful[1]").alias("helpful_1"), \
                                F.collect_list("asin").alias("asin"), \
                                F.collect_list("overall").alias("overall"), \
                                F.collect_list("reviewText").alias("reviewText"), \
                                F.collect_list("summary").alias("summary"), \
                                F.collect_list("reviewerName").alias("reviewerName"), \
                                F.collect_list("unixReviewTime").alias("unixReviewTime"), \
                                F.collect_list("reviewTime").alias("reviewTime")
                                )

In [ ]:
rev_df

In [ ]:
rev_df = rev_df.withColumn("reviewLen", F.udf(lambda a: len(a), T.IntegerType())(F.col("reviewerName")))
rev_df = rev_df.withColumn("reviewSetLen", F.udf(lambda a: len(set(a)), T.IntegerType())(F.col("reviewerName")))
rev_df = rev_df.drop(F.col("reviewLen")).drop(F.col("reviewSetLen"))

In [ ]:
reviewers_set = set([row[0] for row in rev_df.select('reviewerID').collect()])

In [ ]:
len(reviewers_set)

In [ ]:
# rev_df_json = rev_df.toJSON()
# rev_df.coalesce(1).write.format('json').save("/mnt/raid0_ssd_8tb/dataset/Amazon/Refined/Musical_Instruments")

In [ ]:
rev_df

In [ ]:
rev_df = rev_df.groupBy("reviewerID").agg(F.sum("helpful[0]").alias("helpful_0"), \
                                F.sum("helpful[1]").alias("helpful_1"), \
                                F.collect_list("asin").alias("asin"), \
                                F.collect_list("overall").alias("overall"), \
                                F.collect_list("reviewText").alias("reviewText"), \
                                F.collect_list("summary").alias("summary"), \
                                F.collect_list("reviewerName").alias("reviewerName"), \
                                F.collect_list("unixReviewTime").alias("unixReviewTime"), \
                                F.collect_list("reviewTime").alias("reviewTime")
                                )

In [ ]:
# id_list = list(rev_df.select('reviewerID').toPandas()['reviewerID'])

In [ ]:
# s = set()
# dup = list()
# for r_id in id_list:
#     if r_id in s:
#         dup.append(r_id)
#     else:
#         s.add(r_id)

In [ ]:
# print(dup)

In [ ]:
# rev_df.filter(rev_df.reviewerID == 'A1R8OFZY6CYX7G').show()

In [ ]:
rev_df.select(count_distinct("reviewerID")).show()

In [ ]:
rev_df.printSchema()

In [ ]:
meta_df.printSchema()

In [ ]:
meta_df

In [ ]:
meta_df.count()

In [ ]:
upu_groupings = rev_df.groupBy("asin").agg(F.collect_list("reviewerID").alias("reviewerID"))

In [ ]:
upu_groupings

In [ ]:
# upu_groupings = upu_groupings.select("asin", "reviewerID").filter((F.col('reviewerID').isin(reviewers_set)))

In [ ]:
upu_groupings.count()

In [ ]:
def format_date(review_time):
    month, day, year = review_time.replace(",","").split(" ")
    return f"{int(month):02d}-{int(day):02d}-{int(year):04d}"

In [ ]:
format_date_udf = F.udf(lambda x: format_date(x), T.StringType())
usu_groupings = rev_df.withColumn("formatted_date", F.to_date(format_date_udf(F.concat_ws(",", "reviewTime")), "MM-dd-yyyy"))
usu_groupings = usu_groupings.groupBy(F.weekofyear("formatted_date").alias("week"), "overall").agg(F.collect_list("reviewerID").alias("reviewerID"))#.collect()

In [ ]:
usu_groupings.count()

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
words_df = tokenizer.transform(rev_df)

In [ ]:
hashingTF = HashingTF(inputCol="words", outputCol="tf_features")
tf_df = hashingTF.transform(words_df)

In [ ]:
# featurizedData

In [ ]:
idf = IDF(inputCol="tf_features", outputCol="features")
idfModel = idf.fit(tf_df)
tfidf_df = idfModel.transform(tf_df)

In [ ]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="features", outputCol="norm")
tfidf_df= normalizer.transform(tfidf_df).drop(F.col("tf_features")).drop(F.col("features"))

In [ ]:
tfidf_df

In [ ]:
tfidf_sample_df = spark.createDataFrame(tfidf_df.head(10))
# tfidf_sample_df = tfidf_df
# tfidf_sample_df = tfidf_df.sample(fraction=0.3, seed=3)

In [ ]:
tfidf_sample_df.count()

In [ ]:
# features = [row[0] for row in tfidf_df.select('norm').collect()]
# features

In [ ]:
dot_prd = F.udf(lambda x,y: float(x.dot(y)), T.DoubleType())
tfidf_dot_df = tfidf_sample_df.alias("i").join(tfidf_sample_df.alias("j"), F.col("i.norm") < F.col("j.norm"))\
    .select(
        F.col("i.reviewerID").alias("i_reviewerID"),
        F.col("j.reviewerID").alias("j_reviewerID"),
        F.col("i.norm").alias("i"), 
        F.col("j.norm").alias("j"), 
        dot_prd("i.norm", "j.norm").alias("dot"))\
    .sort("i", "j")

# dot_prd = F.udf(lambda x,y: float(x.dot(y)), T.DoubleType())
# tfidf_sample_df.alias("i").join(tfidf_sample_df.alias("j"), F.col("i.norm") < F.col("j.norm"))\
#     .select(
#         F.col("i.norm").alias("i"), 
#         F.col("j.norm").alias("j"), 
#         dot_prd("i.norm", "j.norm").alias("dot"))\
#     .sort("i", "j")

In [ ]:
tfidf_dot_df = tfidf_dot_df.drop(F.col("i")).drop(F.col("j"))

In [ ]:
tfidf_dot_df = tfidf_dot_df.groupBy("i_reviewerID").agg(F.collect_list("j_reviewerID").alias("j_reviewerID"), \
                                F.collect_list("dot").alias("dot")
                                )

In [ ]:
tfidf_dot_df.count()

In [ ]:
# topn = F.udf(lambda a: zip(*heapq.nlargest(5, enumerate(a), key=operator.itemgetter(1)))[0], T.ArrayType(T.IntegerType()))
topn = F.udf(lambda a: sorted(range(len(a)), key=lambda i: a[i])[-5:], T.ArrayType(T.IntegerType()))
tfidf_dot_df = tfidf_dot_df.withColumn("dot", topn(F.col('dot')))
# metadata_tags_df = metadata_tags_df.withColumn('input_ids', F.udf(lambda z: PLMinput_function(z), pst.ArrayType(pst.IntegerType()))(F.col('Synopsis')))

In [ ]:
tfidf_dot_df

In [ ]:
extract_index = F.udf(lambda a, b: [b[i] for i in a], T.ArrayType(T.StringType()))
uvu_grouping = tfidf_dot_df.withColumn("j_reviewerID", extract_index(F.col('dot'), F.col('j_reviewerID'))).drop(F.col("dot"))

In [ ]:
uvu_grouping.collect()

In [ ]:
# tfidf_dot_sample_df = spark.createDataFrame(tfidf_dot_df.head(10))

In [ ]:
# tfidf_dot_sample_df

In [ ]:
# result = data.alias("i").join(data.alias("j"), F.col("i.ID") < F.col("j.ID"))\
#     .select(
#         F.col("i.ID").alias("i"),
#         F.col("j.ID").alias("j"),
#         sim_cos("i.feature", "j.feature").alias("sim_cosine"))\
#     .sort("i", "j")
# result.show()

In [ ]:
tfidf_dot_df.write.mode('Overwrite').json("/mnt/raid0_ssd_8tb/dataset/Amazon/tfidf_dot_df")

In [ ]:
uvu_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/uvu.json")

In [ ]:
uvu_df

In [ ]:
reviewer_set = set()
for u in uvu_df.select('j_reviewerID').collect():
    reviewer_set.update(u[0])
print(reviewer_set)

In [ ]:
reviewer_set.update([row[0] for row in uvu_df.select('i_reviewerID').collect()])

In [ ]:
len(reviewer_set)

In [ ]:
print(reviewer_set)

In [ ]:
##To save in file
with open('/mnt/raid0_ssd_8tb/dataset/Amazon/reviewer_set.txt','wb') as f:
   pickle.dump(reviewer_set, f)

In [ ]:
##To read it again from file
with open('/mnt/raid0_ssd_8tb/dataset/Amazon/reviewer_set.txt','rb') as f:
   reviewer_set = pickle.load(f)

In [ ]:
rev_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Refined/Musical_Instruments/part-00000-d63112ec-17bf-419d-baec-2c7c77685d35-c000.json")

In [ ]:
rev_df

In [ ]:
## Run from here.
with open('/mnt/raid0_ssd_8tb/dataset/Amazon/reviewer_set.pkl', 'rb') as file:
    reviewer_set = pickle.load(file)

In [ ]:
len(reviewer_set)

In [ ]:
# reviewer_set

In [ ]:
index_to_id = dict()
id_to_index = dict()
for i, r in enumerate(reviewer_set):
    index_to_id[i] = r
    id_to_index[r] = i

In [ ]:
with open("/mnt/raid0_ssd_8tb/dataset/Amazon/index_to_id.json", "w") as outfile:
    json.dump(index_to_id, outfile)
with open("/mnt/raid0_ssd_8tb/dataset/Amazon/id_to_index.json", "w") as outfile:
    json.dump(id_to_index, outfile)

In [ ]:
with open('/mnt/raid0_ssd_8tb/dataset/Amazon/index_to_id.json') as json_file:
    index_to_id = json.load(json_file, object_hook=lambda d: {int(k) if k.isdigit() else k: v for k, v in d.items()})

In [ ]:
index_to_id[0]

#### Build a graph

In [103]:
features = sp.load_npz('/mnt/raid0_ssd_8tb/dataset/Amazon/amz_features_25.npz')
# with open('/mnt/raid0_ssd_8tb/dataset/Amazon/amz_features_25.np.npy', 'rb') as f:
#     features = np.load(f, allow_pickle=True)

In [104]:
features.toarray().tolist()[-10:]

[[1.0,
  10.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  -9.999950000398841e-06,
  5.0,
  5.0,
  5.0,
  5.0,
  1.0,
  0.0,
  1.0,
  0.0,
  27.0,
  1.0],
 [1.0,
  10.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  -9.999950000398841e-06,
  5.0,
  5.0,
  5.0,
  5.0,
  3.0,
  0.0,
  1.0,
  0.0,
  41.0,
  1.0],
 [1.0,
  7.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  -9.999950000398841e-06,
  4.0,
  4.0,
  4.0,
  4.0,
  1.0,
  0.0,
  1.0,
  0.0,
  30.0,
  1.0],
 [1.0,
  8.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -9.999950000398841e-06,
  3.0,
  3.0,
  3.0,
  3.0,
  0.0,
  0.0,
  1.0,
  0.0,
  60.0,
  -1.0],
 [1.0,
  11.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  -9.999950000398841e-06,
  1.0,
  1.0,
  1.0,
  1.0,
  21.0,
  0.0,
  1.0,
  0.0,
  20.0,
  1.0],
 [2.0,
  31.0,


In [64]:
#import scipy.sparse as sp
edges = sp.load_npz('/mnt/raid0_ssd_8tb/dataset/Amazon/amz_upsu_adj.npz')

In [105]:
edges.toarray().shape

(11944, 11944)

In [106]:
with open('/mnt/raid0_ssd_8tb/dataset/Amazon/index_to_user.pkl', 'rb') as f:
    index_to_user = pickle.load(f) # index -> user_id
with open('/mnt/raid0_ssd_8tb/dataset/Amazon/user_to_label.pkl', 'rb') as f:
    user_to_label = pickle.load(f) # user_id -> label

In [107]:
labels = []
for i in range(len(index_to_user)):
    labels.append(user_to_label[index_to_user[i]])

In [108]:
# with open('/mnt/raid0_ssd_8tb/dataset/Amazon/reviewer_benign_set.pkl', 'rb') as file:
#     reviewer_benign_set = pickle.load(file)

In [109]:
# len(reviewer_benign_set)

In [110]:
# index_to_label = dict()
# id_to_label = dict()
# for i in index_to_id.keys():
#     if index_to_id[i] in reviewer_benign_set:
#         index_to_label[i] = 0
#         id_to_label[index_to_id[i]] = 0
#     else:
#         index_to_label[i] = 1
#         id_to_label[index_to_id[i]] = 1

In [111]:
# with open("/mnt/raid0_ssd_8tb/dataset/Amazon/index_to_label.json", "w") as outfile:
#     json.dump(index_to_label, outfile)
# with open("/mnt/raid0_ssd_8tb/dataset/Amazon/id_to_label.json", "w") as outfile:
#     json.dump(id_to_label, outfile)

In [112]:
# with open('/mnt/raid0_ssd_8tb/dataset/Amazon/id_to_label.json') as json_file:
#     id_to_label = json.load(json_file, object_hook=lambda d: {int(k) if k.isdigit() else k: v for k, v in d.items()})

In [113]:
# list(id_to_label.values()).count(0)
labels.count(0)

11123

In [114]:
g = dgl.from_scipy(edges)

In [115]:
g.ndata['feature'] = torch.from_numpy(np.float32(features.toarray()))
g.ndata['label'] = torch.from_numpy(np.array(labels))

In [116]:
from dgl.data import FraudYelpDataset, FraudAmazonDataset
dataset = FraudAmazonDataset()
graph = dataset[0]
graph = dgl.to_homogeneous(dataset[0], ndata=['feature', 'label', 'train_mask', 'val_mask', 'test_mask'])

Done loading data from cached files.


In [117]:
graph.ndata['train_mask'].tolist().count(0)

5897

In [118]:
g.ndata['label'].tolist().count(0)

11123

In [119]:
g.ndata['train_mask'] = graph.ndata['train_mask']
g.ndata['val_mask'] = graph.ndata['val_mask']
g.ndata['test_mask'] = graph.ndata['test_mask']

In [120]:
from dgl.data.utils import save_graphs
graph_labels = {"glabel": torch.tensor([0])}
save_graphs("/mnt/raid0_ssd_8tb/dataset/Amazon/graph.bin", [g], graph_labels)

In [121]:
from dgl.data.utils import load_graphs
g, label_dict = load_graphs("/mnt/raid0_ssd_8tb/dataset/Amazon/graph.bin", [0])

In [122]:
g[0].ndata['label'].tolist().count(0)

11123

In [123]:
graph.ndata

{'feature': tensor([[ 1.0000, 26.0000,  0.0000,  ...,  1.0000, 13.0000,  1.0000],
        [ 4.0000, 17.0000,  0.0000,  ...,  0.0000, 45.0000,  1.0000],
        [ 2.0000, 15.0000,  0.0000,  ...,  1.0000, 24.5000,  1.0000],
        ...,
        [ 1.0000, 10.0000,  0.0000,  ...,  1.0000, 15.0000,  1.0000],
        [ 1.0000, 10.0000,  0.0000,  ...,  1.0000, 88.0000,  1.0000],
        [ 1.0000, 10.0000,  0.0000,  ...,  1.0000, 31.0000,  1.0000]]), 'label': tensor([0, 0, 0,  ..., 0, 0, 0]), 'train_mask': tensor([0, 0, 0,  ..., 0, 1, 1], dtype=torch.uint8), 'val_mask': tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8), 'test_mask': tensor([0, 0, 0,  ..., 1, 0, 0], dtype=torch.uint8), '_ID': tensor([    0,     1,     2,  ..., 11941, 11942, 11943]), '_TYPE': tensor([0, 0, 0,  ..., 0, 0, 0])}

In [124]:
g[0].ndata

{'test_mask': tensor([0, 0, 0,  ..., 1, 0, 0], dtype=torch.uint8), 'val_mask': tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8), 'train_mask': tensor([0, 0, 0,  ..., 0, 1, 1], dtype=torch.uint8), 'feature': tensor([[ 1., 15.,  0.,  ...,  0., 18.,  1.],
        [ 1.,  5.,  0.,  ...,  0., 22.,  1.],
        [ 1., 11.,  0.,  ...,  0., 36.,  1.],
        ...,
        [ 1., 10.,  0.,  ...,  0., 15.,  1.],
        [ 1., 10.,  0.,  ...,  0., 88.,  1.],
        [ 1., 10.,  0.,  ...,  0., 31.,  1.]]), 'label': tensor([0, 0, 0,  ..., 0, 0, 0])}

In [125]:
graph

Graph(num_nodes=11944, num_edges=9557648,
      ndata_schemes={'feature': Scheme(shape=(25,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.uint8), 'val_mask': Scheme(shape=(), dtype=torch.uint8), 'test_mask': Scheme(shape=(), dtype=torch.uint8), '_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), '_TYPE': Scheme(shape=(), dtype=torch.int64)})

In [126]:
g

[Graph(num_nodes=11944, num_edges=9734218,
       ndata_schemes={'test_mask': Scheme(shape=(), dtype=torch.uint8), 'val_mask': Scheme(shape=(), dtype=torch.uint8), 'train_mask': Scheme(shape=(), dtype=torch.uint8), 'feature': Scheme(shape=(25,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={})]